In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pickle

In [34]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Preprocessing 

In [35]:
## first three features are not useful
df = df.drop(['CustomerId', 'Surname', 'RowNumber'], axis=1)

In [36]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [37]:
## Geagraphy and Gender are categorical features

label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [38]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [39]:
## One hot Encoding for Geography

from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
geo_encoder = onehotencoder.fit_transform(df[['Geography']])
geo_encoder


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [40]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [41]:
##get features name

geo_encoder_feature_names = onehotencoder.get_feature_names_out(['Geography'])
geo_encoder_feature_names

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [42]:
#making a dataframe
geo_df = pd.DataFrame(geo_encoder.toarray(), columns=geo_encoder_feature_names)

In [43]:
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [44]:
## combining one hot encoded features with original dataframe

df = pd.concat([df.drop('Geography', axis=1), geo_df], axis=1)

In [45]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [46]:
## save the encoder and scaler

with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('onehotencoder_geography.pkl', 'wb') as f:
    pickle.dump(onehotencoder, f)



In [47]:
## divide data set into features and target
X = df.drop('Exited', axis=1)
y = df['Exited']


#split into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## scaling the features

scaler = StandardScaler()

X_train  = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [48]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

## ANN Implementation

In [49]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [50]:
import datetime

In [51]:
## build our ANN model
model = Sequential(
    [
        Dense(64,activation = 'relu', input_shape=(X_train.shape[1],)),
        Dense(32,activation = 'relu'),
        Dense(1,activation = 'relu')
    ]
)

/Users/kaccatur/Documents/Ann_Project/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [52]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [54]:
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [66]:
## set up tensorboard
log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [67]:
## Set up Early stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [68]:
## Train the model

history = model.fit(
    X_train, y_train, validation_data = (X_test,y_test),epochs=100, 
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step - accuracy: 0.8502 - loss: 0.3674 - val_accuracy: 0.8665 - val_loss: 0.3523
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - accuracy: 0.8532 - loss: 0.3646 - val_accuracy: 0.8575 - val_loss: 0.3679
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step - accuracy: 0.8566 - loss: 0.3572 - val_accuracy: 0.8095 - val_loss: 0.4327
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step - accuracy: 0.8359 - loss: 0.4051 - val_accuracy: 0.8325 - val_loss: 0.4167
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 540us/step - accuracy: 0.8459 - loss: 0.3957 - val_accuracy: 0.8530 - val_loss: 0.3734
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.8565 - loss: 0.3640 - val_accuracy: 0.8625 - val_loss: 0.3794
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step - accuracy: 0.8423 - loss: 0.4422 - val_accuracy: 0.8415 - val_loss: 0.7790
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - accuracy: 0.8495 - loss: 0

In [69]:
model.save('churn_model.h5')

In [70]:
%load_ext tensorflow

The tensorflow module is not an IPython extension.


In [71]:
!pip install tensorboard

In [76]:
%tensorboard --logdir log/fit

UsageError: Line magic function `%tensorboard` not found.
